In [ ]:
from collections import defaultdict
import pandas as pd

You can find the data used below on this [link](https://www.kaggle.com/datasets/amiramosa99/arabic-tweets-in-different-dialects-clean-data).

In [ ]:
corpora_1 = pd.read_csv("clean_data.csv", encoding='utf-8').loc[:5000, 'tweets']
corpora_2 = pd.read_csv("clean_data.csv", encoding='utf-8').loc[5000:10000, "tweets"]

In [ ]:
# # s = "Muhammad loves Allah and wants to stay on his stright path May Allah accepts Muhammad good deeds What Allah wants happens"
# s = "و قبر حرب ب مكان قفر و ليس قرب قبر حرب قبر أتيت على قبر حرب ف بكيت حرب و أتيت على قبر جدتي بعدها"

def ngrams_analysis(corp):
    total_unigrams = []
    for s in corp:
        unigrams = s.split()
        total_unigrams += unigrams
    # print(total_unigrams)

    unigram_count = defaultdict(int)
    for w in total_unigrams:
        unigram_count[w]+=1
    # print(dict(sorted(unigram_count.items(), key=lambda item: item[1])))

    unigrams_prob = {w: c/len(total_unigrams) for w,c in unigram_count.items()}
    # print(dict(sorted(unigrams_prob.items(), key=lambda item: item[1], reverse=True)))

    bigram_count = defaultdict(int)
    for i in range(len(total_unigrams)-1):
        w1, w2 = total_unigrams[i], total_unigrams[i+1]
        bigram_count[(w1, w2)] += 1
    # print(dict(sorted(bigram_count.items(), key=lambda item: item[1], reverse=True)))

    bigrams_prob = {bigram: c/unigram_count[bigram[0]] for bigram,c in bigram_count.items()}
    # print(bigrams_prob)

    return {"unigram_count": unigram_count,
            "unigrams_prob": unigrams_prob,
            "bigram_count": bigram_count,
            "bigrams_prob": bigrams_prob}

In [ ]:
# corp1 analysis
model_1 = ngrams_analysis(corpora_1)

# corp2 analysis
model_2 = ngrams_analysis(corpora_2)

In [ ]:
import random

In [ ]:
def generate(prompt:str, model:dict, max_tokens:int) -> str:

    for i in range(max_tokens):

        last_word = prompt.split()[-1]
        candidates = []
        probs = []

        for bigram,prob in model["bigrams_prob"].items():

            if bigram[0] == last_word:
                candidates.append(bigram[1])
                probs.append(prob)

        if not candidates:
            break

        total_prob = sum(probs)
        normalized_probs = [p/total_prob for p in probs]
        next_word = random.choices(candidates, weights=normalized_probs, k=1)[0]

        prompt += " " + next_word

    return prompt

In [ ]:
import numpy as np

In [ ]:
def perplexity(text: str, model: dict, smoothing_value = 1e-10) -> float:

    tokens = text.split()
    n = len(tokens)

    in_bigrams = [ (tokens[i], tokens[i+1]) for i in range(n-1) ]

    bigrams_prob = model["bigrams_prob"]

    in_probs = []

    for in_bigram in in_bigrams:

        if in_bigram in bigrams_prob:
            in_probs.append(bigrams_prob[in_bigram])
        else:
            in_probs.append(smoothing_value)

    in_probs = np.array(in_probs, dtype="float32")

    log_sum = np.sum(np.log(in_probs))
    perplexity = np.exp( -log_sum/n )

    return perplexity

In [ ]:
model_1_generate = generate("محمد", model_1, max_tokens=50)
model_1_generate

'محمد المهنا شكد اضوج ادخل تويتر شسوي لعد واحد شايف شي ميعجبني ما انتي اي ميخالف شوفني خل ايرفعوه وي مداستوعب اي شي تطلع فيمنست قبل صايره احب خصوصا بالاماكن الي راحو ونحجي عليهم كلهم طالعين لشغل ما عندنا عبالك طالعه من النتايج خلص وتذكرت ابنها لقصر واخذ اول شي واله'

In [ ]:
perplexity(model_1_generate, model_1)

11.969854419973643

In [ ]:
model_2_generate = generate("محمد", model_2, max_tokens=50)
model_2_generate

'محمد ليش الجذب كلشي مفتهمت منها هذا الحكي ما كواني مالي شغل احزاب اليمين المتطرف دا يتظاهرن لو نكول اشمانكول ما شاله واذا نريد نتخلص من تلبس ملابس شرطه هم بمدينه عراقيه وصداميه علي الردود تعرفهم تريد شي تهم ايران ليش الغيبه مثلا اي هزه خلي يرد عليك وعلي يوري البغل'

In [ ]:
perplexity(model_2_generate, model_2)

11.854948901283919

In [ ]:
dict(sorted(model_1["unigram_count"].items(), key=lambda item: item[1], reverse=True))

{'من': 1037,
 'ما': 740,
 'بس': 649,
 'الي': 596,
 'مو': 564,
 'علي': 483,
 'اني': 376,
 'واله': 360,
 'اله': 358,
 'ه': 301,
 'لا': 289,
 'لو': 266,
 'يعني': 256,
 'هذا': 247,
 'هاي': 246,
 'اي': 245,
 'اذا': 237,
 'شنو': 227,
 'كل': 224,
 'شي': 215,
 'لان': 212,
 'هو': 205,
 'انت': 194,
 'ليش': 194,
 'حتي': 190,
 'هسه': 189,
 'واحد': 185,
 'يا': 182,
 'راح': 163,
 'هيج': 160,
 'هم': 148,
 'او': 148,
 'ولا': 143,
 'ع': 142,
 'بعد': 139,
 'اكو': 120,
 'انا': 117,
 'بيه': 110,
 'شلون': 109,
 'وين': 108,
 'في': 108,
 'عن': 101,
 'ان': 99,
 'منو': 99,
 'احنه': 98,
 'عليه': 94,
 'الناس': 94,
 'ابو': 93,
 'مثل': 92,
 'العراق': 90,
 'بيها': 89,
 'مع': 88,
 'ماكو': 88,
 'عندي': 86,
 'لازم': 84,
 'مال': 84,
 'صار': 83,
 'كلش': 82,
 'يوم': 80,
 'و': 77,
 'حلو': 76,
 'هي': 75,
 'اعرف': 74,
 'غير': 74,
 'يصير': 74,
 'انتي': 74,
 'انو': 70,
 'مره': 69,
 'قبل': 68,
 'صح': 66,
 'رح': 66,
 'اليوم': 65,
 'اول': 63,
 'احد': 63,
 'جان': 63,
 'زين': 62,
 'اكيد': 62,
 'عنده': 61,
 'عليك': 60,
 'صدك': 59,


In [ ]:
dict(sorted(model_1["unigrams_prob"].items(), key=lambda item: item[1], reverse=True))

{'من': 0.018484189511960358,
 'ما': 0.013190260596770169,
 'بس': 0.01156821503689708,
 'الي': 0.010623507183344622,
 'مو': 0.010053117535916723,
 'علي': 0.008609318740864853,
 'اني': 0.006702078357277815,
 'واله': 0.006416883533563866,
 'اله': 0.0063812341805996224,
 'ه': 0.005365227621118677,
 'لا': 0.005151331503333215,
 'لو': 0.004741363944244412,
 'يعني': 0.0045631171794231935,
 'هذا': 0.004402695091084097,
 'هاي': 0.004384870414601975,
 'اي': 0.004367045738119853,
 'اذا': 0.004224448326262878,
 'شنو': 0.00404620156144166,
 'كل': 0.003992727531995294,
 'شي': 0.0038323054436561975,
 'لان': 0.003778831414209832,
 'هو': 0.003654058678834979,
 'انت': 0.003457987237531639,
 'ليش': 0.003457987237531639,
 'حتي': 0.0033866885316031514,
 'هسه': 0.0033688638551210297,
 'واحد': 0.003297565149192542,
 'يا': 0.0032440911197461767,
 'راح': 0.0029054222665858614,
 'هيج': 0.002851948237139496,
 'هم': 0.0026380521193540337,
 'او': 0.0026380521193540337,
 'ولا': 0.0025489287369434245,
 'ع': 0.002531

In [ ]:
dict(sorted(model_1["bigram_count"].items(), key=lambda item: item[1], reverse=True))

{('شاء', 'اله'): 47,
 ('ان', 'شاء'): 41,
 ('اي', 'واله'): 38,
 ('بس', 'مو'): 26,
 ('ه', 'ه'): 26,
 ('اهم', 'شي'): 24,
 ('مو', 'بس'): 22,
 ('بس', 'اني'): 22,
 ('اكثر', 'من'): 21,
 ('ما', 'اعرف'): 21,
 ('اني', 'هم'): 20,
 ('الي', 'ما'): 20,
 ('ما', 'ادري'): 19,
 ('الناس', 'الي'): 19,
 ('اكو', 'ناس'): 18,
 ('مع', 'الاسف'): 17,
 ('اول', 'مره'): 17,
 ('اني', 'ما'): 17,
 ('كل', 'ما'): 16,
 ('ليش', 'ما'): 16,
 ('كل', 'شي'): 14,
 ('هو', 'الي'): 14,
 ('احلي', 'من'): 14,
 ('بالف', 'خير'): 13,
 ('مو', 'اني'): 13,
 ('هذا', 'الي'): 13,
 ('لا', 'واله'): 13,
 ('يعني', 'من'): 13,
 ('من', 'كل'): 13,
 ('مثل', 'ما'): 13,
 ('اذا', 'ما'): 13,
 ('اله', 'يبارك'): 13,
 ('لحد', 'الان'): 12,
 ('ه', 'واله'): 12,
 ('بس', 'من'): 12,
 ('ه', 'اي'): 12,
 ('لا', 'مو'): 12,
 ('بس', 'هو'): 12,
 ('من', 'الصبح'): 12,
 ('بس', 'ما'): 12,
 ('كل', 'يوم'): 12,
 ('واحد', 'من'): 12,
 ('بس', 'انت'): 11,
 ('من', 'وره'): 11,
 ('من', 'زمان'): 11,
 ('اله', 'يرحمه'): 11,
 ('شنو', 'يعني'): 11,
 ('لو', 'شنو'): 11,
 ('اول', 'شي'): 11,
 (

In [ ]:
model_1["bigrams_prob"]

{('لكن', 'بالنهايه'): 0.022222222222222223,
 ('بالنهايه', 'ينتفض'): 0.3333333333333333,
 ('ينتفض', 'يغير'): 1.0,
 ('يغير', 'يعني'): 0.5,
 ('يعني', 'هذا'): 0.01953125,
 ('هذا', 'محسوب'): 0.004048582995951417,
 ('محسوب', 'علي'): 0.5,
 ('علي', 'البشر'): 0.002070393374741201,
 ('البشر', 'حيونه'): 0.09090909090909091,
 ('حيونه', 'وحشيه'): 0.3333333333333333,
 ('وحشيه', 'وتطلبون'): 1.0,
 ('وتطلبون', 'من'): 1.0,
 ('من', 'الغرب'): 0.0009643201542912247,
 ('الغرب', 'يحترمكم'): 0.25,
 ('يحترمكم', 'ويءمن'): 1.0,
 ('ويءمن', 'بدينكم'): 1.0,
 ('بدينكم', 'ولاينعتكم'): 1.0,
 ('ولاينعتكم', 'بالارهاب'): 1.0,
 ('بالارهاب', 'مبين'): 1.0,
 ('مبين', 'من'): 0.4,
 ('من', 'كلامه'): 0.0009643201542912247,
 ('كلامه', 'خليجي'): 0.3333333333333333,
 ('خليجي', 'يسلملي'): 0.5,
 ('يسلملي', 'مرورك'): 1.0,
 ('مرورك', 'وروحك'): 0.3333333333333333,
 ('وروحك', 'الحلوه'): 1.0,
 ('الحلوه', 'وين'): 0.05555555555555555,
 ('وين', 'هل'): 0.009259259259259259,
 ('هل', 'الغيبه'): 0.03508771929824561,
 ('الغيبه', 'اخ'): 0.5,
 ('اخ